In [45]:
import requests
from requests_futures.sessions import FuturesSession
import json

para = [{'pincode':'444001',
           'date': '4-5-2021' },
           {'pincode':'444002',
           'date': '4-5-2021' }
           {'pincode':'111001',
           'date': '4-5-2021' }]

SyntaxError: invalid syntax (<ipython-input-45-e6e1014bd1df>, line 9)

In [43]:
%%timeit -r 1
for pa in para:
    requests.get(url="https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByPin",params=pa)

412 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [44]:
%%timeit -r 1

with FuturesSession() as session:
    futures = [session.get(url) for url in urls]
    for future in futures:
        future.result()

NameError: name 'FuturesSession' is not defined